<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI_AI_AGENT_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re # Import regex for robust parsing
import json # For safer JSON parsing of tool inputs
from enum import Enum
from typing import List, Dict, Any, Optional

# --- Google Colab / Gemini API Imports ---
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not running in Google Colab. Ensure 'GEMINI' environment variable is set for the API key.")
    GOOGLE_API_KEY = os.getenv('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("GEMINI API Key not found. Please set it as an environment variable or Colab Secret.")
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using environment variable.")


# --- Configuration ---
class AgentConfig:
    LLM_PROVIDER: str = "google"
    LLM_MODEL_NAME: str = "gemini-1.5-flash"
    VECTOR_DB_TYPE: str = "conceptual_vector_db"
    VECTOR_DB_CONNECTION_STRING: str = "http://localhost:8000"
    DOC_CHUNKING_STRATEGY: str = "recursive_character"
    MEMORY_STORAGE_TYPE: str = "in_memory_list"
    MEMORY_EXPIRATION_SECONDS: int = 3600
    PLANNING_FRAMEWORK: str = "conceptual_planning"
    TOOL_REGISTRY: Dict[str, Any] = {}
    INTERFACE_TYPE: str = "cli"


# --- 1. Core Model Layer (LLM Abstraction) ---
class LLMService:
    def __init__(self, config: AgentConfig):
        self.config = config
        self.llm_client = self._initialize_llm_client()

    def _initialize_llm_client(self):
        print(f"Initializing Google GenAI client with model: {self.config.LLM_MODEL_NAME}")
        return genai.GenerativeModel(self.config.LLM_MODEL_NAME)

    def generate_response(self, prompt: str, context: Optional[str] = None, tools_schema: Optional[List[Dict]] = None) -> str:
        full_prompt = f"Context: {context}\nPrompt: {prompt}" if context else prompt
        print(f"\n--- LLM Input ---\n{full_prompt}\n---")

        messages = [{"role": "user", "parts": [full_prompt]}]

        try:
            configured_tools = []
            if tools_schema:
                for tool_spec in tools_schema:
                    if "name" in tool_spec and "parameters" in tool_spec:
                        tool_declaration = genai.types.Tool(
                            function_declarations=[
                                genai.types.FunctionDeclaration(
                                    name=tool_spec["name"],
                                    description=tool_spec.get("description", ""),
                                    parameters=tool_spec["parameters"]
                                )
                            ]
                        )
                        configured_tools.append(tool_declaration)

            if configured_tools:
                response = self.llm_client.generate_content(messages, tools=configured_tools)
            else:
                response = self.llm_client.generate_content(messages)

            # Process response
            if response.parts:
                return response.text
            elif response.candidates and response.candidates[0].function_calls:
                function_calls = response.candidates[0].function_calls
                tool_call = function_calls[0]
                # Return a distinct string format for tool calls that can be easily parsed
                return f"TOOL_CALL:{tool_call.name}({json.dumps(tool_call.args)})"
            else:
                print("LLM generated no text or direct function call.")
                return "No response generated by LLM."
        except Exception as e:
            print(f"Error generating content from LLM: {e}")
            return f"Error from LLM: {e}"


# --- 2. Retrieval Layer (RAG & Vector Store) ---
class VectorStore:
    def __init__(self, config: AgentConfig):
        self.config = config
        print(f"Initializing Conceptual Vector Store ({self.config.VECTOR_DB_TYPE}).")

    def add_documents(self, texts: List[str], metadatas: Optional[List[Dict]] = None):
        print(f"Adding {len(texts)} conceptual documents to vector store.")

    def retrieve(self, query: str, top_k: int = 2) -> List[str]:
        print(f"Retrieving top {top_k} conceptual documents for query: '{query}'")
        if "flight" in query.lower():
            return [
                "Document 1: General flight regulations and international travel advisories.",
                "Document 2: Information about Montreal-Trudeau International Airport (YUL)."
            ]
        elif "france" in query.lower():
            return [
                "Document 3: France travel guide: attractions and visa requirements.",
                "Document 4: Capital cities of European countries."
            ]
        return [f"Retrieved document for '{query}' (conceptual)"]

class RAGSystem:
    def __init__(self, vector_store: VectorStore, llm_service: LLMService, config: AgentConfig):
        self.vector_store = vector_store
        self.llm_service = llm_service
        self.config = config

    def retrieve_and_augment(self, query: str) -> str:
        retrieved_docs = self.vector_store.retrieve(query)
        context = "\n".join(retrieved_docs)
        print(f"RAG: Augmented context with {len(retrieved_docs)} documents.")
        return context


# --- 3. Memory Layer (Short-term / Long-term / Semantic) ---
class Memory:
    def __init__(self, config: AgentConfig):
        self.config = config
        self.short_term_memory: List[str] = []
        self.long_term_memory: Dict[str, Any] = {}
        print(f"Initializing Conceptual Memory System ({self.config.MEMORY_STORAGE_TYPE}).")

    def add_short_term(self, entry: str):
        self.short_term_memory.append(entry)
        print(f"Added to short-term memory: '{entry}'")

    def get_short_term(self) -> List[str]:
        return self.short_term_memory

    def add_long_term(self, key: str, value: Any):
        self.long_term_memory[key] = value
        print(f"Added to long-term memory: '{key}'")

    def get_long_term(self, key: str) -> Optional[Any]:
        return self.long_term_memory.get(key)

    def retrieve_semantic_memory(self, query: str) -> List[str]:
        print(f"Performing conceptual semantic memory retrieval for: '{query}'")
        return [f"Semantic memory relevant to '{query}' (conceptual)"]


# --- 4. Planning Layer (Conceptual) ---
class AgentPlanner:
    def __init__(self, llm_service: LLMService, config: AgentConfig):
        self.llm_service = llm_service
        self.config = config
        print(f"Initializing Conceptual Planner with framework: {self.config.PLANNING_FRAMEWORK}")

    def create_plan(self, objective: str, tools_available: List[Dict]) -> List[str]:
        tool_names = [tool["name"] for tool in tools_available]

        prompt = (
            f"You are an AI agent. Given the objective: '{objective}', "
            f"and the following tools you can potentially use: {', '.join(tool_names)}.\n"
            "Please outline a detailed, step-by-step plan to achieve this objective. "
            "Think step-by-step. Start your response with 'Plan:'"
        )

        plan_suggestion = self.llm_service.generate_response(prompt)

        if plan_suggestion.startswith("Plan:"):
            plan_suggestion = plan_suggestion[len("Plan:"):].strip()

        steps = [step.strip() for step in plan_suggestion.split('\n') if step.strip()]

        if not steps:
            steps = [f"Step 1: Analyze objective '{objective}'",
                     f"Step 2: Use available tools if necessary",
                     f"Step 3: Formulate final answer"]

        print(f"Generated conceptual plan for '{objective}': {steps}")
        return steps

    def decompose_task(self, task: str) -> List[str]:
        print(f"Decomposing conceptual task: '{task}'")
        return [f"Conceptual Subtask A for '{task}'", f"Conceptual Subtask B for '{task}'"]


# --- 5. Execution Layer (Tool calling / API / File I/O) ---
class Tool:
    def __init__(self, name: str, description: str, parameters: Dict):
        self.name = name
        self.description = description
        self.parameters = parameters

    def execute(self, **kwargs) -> Any:
        raise NotImplementedError("Each tool must implement its own execute method.")

    def get_tool_spec(self) -> Dict:
        return {
            "name": self.name,
            "description": self.description,
            "parameters": self.parameters
        }

class SearchTool(Tool):
    def __init__(self):
        super().__init__(
            name="SearchTool",
            description="A tool to search the internet for information.",
            parameters={
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query."}
                },
                "required": ["query"]
            }
        )

    def execute(self, query: str) -> str:
        print(f"Executing SearchTool with query: '{query}'")
        if "capital of france" in query.lower():
            return "Paris is the capital of France."
        elif "flight booking websites" in query.lower(): # Added for flight planning context
            return "Expedia, Kayak, Google Flights, Skyscanner, Booking.com, Momondo."
        return f"Conceptual search results for '{query}': Example finding related to your query."

class FileIOTool(Tool):
    def __init__(self):
        super().__init__(
            name="FileIOTool",
            description="A tool to read from and write to files.",
            parameters={
                "type": "object",
                "properties": {
                    "action": {"type": "string", "enum": ["read", "write"], "description": "The file operation (read or write)."},
                    "path": {"type": "string", "description": "The file path."},
                    "content": {"type": "string", "description": "Content to write (required for 'write' action)."}
                },
                "required": ["action", "path"]
            }
        )

    def execute(self, action: str, path: str, content: Optional[str] = None) -> str:
        print(f"Executing FileIOTool: {action} on {path}")
        if action == "read":
            # Conceptual file read
            if "my_data.txt" in path: # Check for the specific file name used in the demo
                return "Content of my_data.txt: 'Hello AI Agent!'" # Simulate content written by agent
            return f"Could not conceptually read from {path}. File not found or empty."
        elif action == "write":
            # Conceptual file write
            print(f"Conceptually writing '{content}' to {path}.")
            # In a real app, this would actually write to a file.
            return f"Wrote '{content}' to {path} (conceptually)."
        else:
            return "Invalid file I/O action."

class AgentExecutor:
    def __init__(self, tools_registry: Dict[str, Tool]):
        self.tools_registry = tools_registry
        print(f"Executor initialized with {len(tools_registry)} tools.")

    def execute_action(self, action_type: str, details: Dict[str, Any]) -> Any:
        print(f"\n--- Executing Action: {action_type} ---")
        if action_type == "tool_call":
            tool_name = details.get("tool_name")
            tool_args = details.get("tool_args", {})
            if tool_name in self.tools_registry:
                tool = self.tools_registry[tool_name]
                try:
                    result = tool.execute(**tool_args)
                    print(f"Tool '{tool_name}' executed successfully. Result: {result}")
                    return result
                except Exception as e:
                    print(f"Error executing tool '{tool_name}': {e}")
                    return f"Error: {e}"
            else:
                print(f"Error: Tool '{tool_name}' not found.")
                return f"Error: Tool '{tool_name}' not found."
        elif action_type == "api_call":
            print(f"Making conceptual API call to: {details.get('endpoint')}")
            return "API call simulated result."
        else:
            print(f"Unknown action type: {action_type}")
            return "Unknown action."


# --- 6. Interaction Layer (ReAct / GUI / Multimodal - Conceptual) ---
class AgentInteraction:
    def __init__(self, llm_service: LLMService, planner: AgentPlanner, executor: AgentExecutor, memory: Memory, rag_system: RAGSystem, available_tools_for_llm: List[Dict]):
        self.llm_service = llm_service
        self.planner = planner
        self.executor = executor
        self.memory = memory
        self.rag_system = rag_system
        self.available_tools_for_llm = available_tools_for_llm
        print("Interaction Layer (ReAct loop) initialized.")

    def run_react_loop(self, initial_query: str):
        print(f"\n--- Starting ReAct Loop for query: '{initial_query}' ---")

        current_thought = f"The user's query is: '{initial_query}'. I need to determine the best course of action."
        observation = ""
        max_iterations = 7

        for i in range(max_iterations):
            print(f"\n--- ReAct Iteration {i+1} ---")
            self.memory.add_short_term(f"ReAct Iteration {i+1}: Thought: {current_thought}, Observation: {observation}")

            react_prompt_template = """
You are an AI agent. Respond in a ReAct format:
Thought: Your reasoning process, explaining your steps and why.
Action: <tool_name>
Action Input: <JSON_arguments_for_tool>
OR
Final Answer: <Your final response to the user query>

Current Thought: {current_thought}
Observation from previous action: {observation}

Your goal is to fully answer the user's initial query.
"""
            llm_prompt = react_prompt_template.format(
                current_thought=current_thought,
                observation=observation
            )

            llm_response_raw = self.llm_service.generate_response(llm_prompt, tools_schema=self.available_tools_for_llm)
            print(f"Raw LLM Response: {llm_response_raw}") # Debugging: see exactly what the LLM returned

            # --- START OF IMPROVED PARSING LOGIC ---
            parsed_action = None
            parsed_input = None
            final_answer_match = re.search(r"Final Answer:\s*(.*)", llm_response_raw, re.DOTALL)
            tool_call_match = re.search(r"TOOL_CALL:([a-zA-Z0-9_]+)\((.*)\)", llm_response_raw)

            if final_answer_match:
                final_answer = final_answer_match.group(1).strip()
                print(f"Agent's Final Answer: {final_answer}")
                self.memory.add_short_term(f"Final Answer: {final_answer}")
                return final_answer
            elif tool_call_match:
                tool_name = tool_call_match.group(1).strip()
                args_json_str = tool_call_match.group(2).strip()
                try:
                    tool_args = json.loads(args_json_str)
                    parsed_action = tool_name
                    parsed_input = tool_args
                except json.JSONDecodeError as e:
                    print(f"JSON decoding error for tool arguments: {e}. Raw: {args_json_str}")
                    observation = f"Error: LLM provided invalid JSON for tool arguments: {e}. Raw: {args_json_str}"
                    current_thought = f"Error in parsing tool arguments. Re-evaluating. Error: {observation}"
            else:
                # If not a final answer or structured tool call, assume it's just a thought or malformed output
                print(f"LLM output was not a clear Final Answer or TOOL_CALL pattern. Processing as thought.")
                current_thought = f"The LLM provided: '{llm_response_raw}'. I need to continue my reasoning towards a final answer, possibly involving tools."
                observation = "No specific tool action was indicated by the LLM in the last step. Continue reasoning."

            # --- END OF IMPROVED PARSING LOGIC ---

            if parsed_action:
                print(f"LLM decided to call tool: {parsed_action} with args: {parsed_input}")
                action_result = self.executor.execute_action(
                    action_type="tool_call",
                    details={"tool_name": parsed_action, "tool_args": parsed_input}
                )
                observation = f"Result of {parsed_action} call: {action_result}"
                current_thought = f"I have executed the {parsed_action} tool and received the following observation. Now I need to process this observation and decide the next step or if I have a final answer."
            elif not final_answer_match: # Only if a final answer wasn't found and no tool was parsed
                # If the LLM still isn't giving a tool call or final answer, maybe it's just thinking.
                # The 'observation' will guide the next step.
                pass # The `current_thought` and `observation` are already set above in the else block.

            if i == max_iterations - 1:
                print("Max iterations reached. Could not find a final answer.")
                final_answer = f"Could not determine a final answer within {max_iterations} iterations. Last thought: {current_thought}. Last observation: {observation}"
                self.memory.add_short_term(f"Failed to find final answer: {final_answer}")
                return final_answer

        return "Agent could not reach a conclusion." # Fallback return


# --- Main Agent Class (Orchestrates all layers) ---
class GeminiFlightPlanningAgent:
    def __init__(self, config: AgentConfig):
        print("\n--- Initializing Gemini Flight Planning Agent ---")
        self.config = config

        self.llm_service = LLMService(self.config)

        self.tools_registry = {
            "SearchTool": SearchTool(),
            "FileIOTool": FileIOTool(),
        }
        self.executor = AgentExecutor(self.tools_registry)

        available_tools_for_llm = [tool.get_tool_spec() for tool in self.tools_registry.values()]

        self.vector_store = VectorStore(self.config)
        self.rag_system = RAGSystem(self.vector_store, self.llm_service, self.config)
        self.memory = Memory(self.config)
        self.planner = AgentPlanner(self.llm_service, self.config)

        self.interaction_layer = AgentInteraction(
            llm_service=self.llm_service,
            planner=self.planner,
            executor=self.executor,
            memory=self.memory,
            rag_system=self.rag_system,
            available_tools_for_llm=available_tools_for_llm
        )
        print("--- Gemini Flight Planning Agent Initialized ---")

    def plan_flight(self, destination: str, date: str, passengers: int):
        print(f"\n--- Agent received flight planning request ---")
        query = f"Plan a flight to {destination} on {date} for {passengers} passengers."
        self.memory.add_short_term(f"Flight Planning Request: {query}")

        rag_context = self.rag_system.retrieve_and_augment(f"Flight information for {destination}")
        print(f"RAG Context for planning: {rag_context}")

        flight_plan_steps = self.planner.create_plan(
            objective=query,
            tools_available=[tool.get_tool_spec() for tool in self.tools_registry.values()]
        )
        self.memory.add_short_term(f"Initial Flight Plan: {flight_plan_steps}")

        print("\n--- Initiating ReAct loop for detailed planning and execution ---")
        final_result = self.interaction_layer.run_react_loop(query)

        print(f"\n--- Flight Planning Process Concluded ---")
        print(f"Final Outcome: {final_result}")
        print(f"Short-term memory log: {self.memory.get_short_term()}")
        return final_result


# --- Main Execution ---
if __name__ == "__main__":
    agent_config = AgentConfig()
    flight_agent = GeminiFlightPlanningAgent(agent_config)

    print("\n" + "="*50)
    print("DEMO 1: Flight Planning Query")
    print("="*50)
    flight_agent.plan_flight(destination="Paris", date="2025-09-15", passengers=2)

    print("\n\n" + "="*50)
    print("DEMO 2: General Knowledge Query (using SearchTool)")
    print("="*50)
    flight_agent.interaction_layer.run_react_loop("What is the capital of France?")

    print("\n\n" + "="*50)
    print("DEMO 3: File I/O Query (using FileIOTool)")
    print("="*50)
    flight_agent.interaction_layer.run_react_loop("Can you write 'Hello AI Agent!' into a file named 'my_data.txt'?")

Google Generative AI configured successfully using Colab Secrets.

--- Initializing Gemini Flight Planning Agent ---
Initializing Google GenAI client with model: gemini-1.5-flash
Executor initialized with 2 tools.
Initializing Conceptual Vector Store (conceptual_vector_db).
Initializing Conceptual Memory System (in_memory_list).
Initializing Conceptual Planner with framework: conceptual_planning
Interaction Layer (ReAct loop) initialized.
--- Gemini Flight Planning Agent Initialized ---

DEMO 1: Flight Planning Query

--- Agent received flight planning request ---
Added to short-term memory: 'Flight Planning Request: Plan a flight to Paris on 2025-09-15 for 2 passengers.'
Retrieving top 2 conceptual documents for query: 'Flight information for Paris'
RAG: Augmented context with 2 documents.
RAG Context for planning: Document 1: General flight regulations and international travel advisories.
Document 2: Information about Montreal-Trudeau International Airport (YUL).

--- LLM Input ---
Y